А теперь задачка под номером два - анализ финансовых рядов. Как вы наверняка знаете, самой популярной криптовалютой является Bitcoin, стоимость которого уже превысила $15k. Однако, в мире существуют еще несколько тысяч альтернативных криптовалют, бОльшая часть из которых ничего особенного из себя не представляют и даже динамика их цен совпадает с флагманским биткоином. Вашей задачей будет найти криптовалюты, которые не будут попадать в кластер биткоина на разных временных промежутках
Установите библиотеку cryptocompare (https://pypi.org/project/cryptocompare/)
Используя cryptocompare.get_coin_list(format=True) получите список из всех торгующихся криптовалют (должно получиться больше 5600). Если желания долго скачивать данные нет, то можно случайным образом отобрать штучек 100 валют из этого списка, только убедитесь, что биткоин (символ - BTC) в него всё ещё входит.
Теперь скачайте цены закрытия для имеющихся криптовалют по трем разным временным промежуткам:
За последние 30 дней с дневной частотой - cryptocompare.get_historical_price_day(ticker, curr='USD', limit=30)
За последние 3 дня с часовой частотой - cryptocompare.get_historical_price_hour(ticker, curr='USD', limit=3*24)
За последний час с минутной частотой - cryptocompare.get_historical_price_minute(ticker, curr='USD', limit=60)
А теперь самое интересное - проведите кластерный анализ для каждой из временных групп и найдите валюты, которые попали в кластеры, отличные от кластера с биткоином, попробуйте как “классическую” кластеризацию, так и DTW.


# Формирование списка токенов и загрузка данных

In [48]:
import pandas as pd
import cryptocompare as cc
from tqdm.auto import tqdm
import matplotlib.pyplot as plt 
import random

In [83]:
cc.get_avg('BTC', 'USDT')['PRICE']

18929.62

In [84]:
crypto_list = cc.get_coin_list(format=True)
len(crypto_list)


8089

In [86]:
crypto_list[0]

'BTCD'

In [92]:
alts_list = []

In [120]:
# приходилось запускать несколько раз - блокировали аккаунт
# отбираем только те монеты цена которых больше 0.1 USDT
for i,item in enumerate(crypto_list[6114:]):
    avg_item = cc.get_avg(item, 'USDT')
    if avg_item is not None:
        price = avg_item["PRICE"]
        print(f'{i}__{item} = {price}')
        if price > 0.1:
            alts_list.append(item)

len(alts_list)

0__GMTT = 0.1009
[ERROR] CCCAGG market does not exist for this coin pair (MYCE-USDT)
[ERROR] CCCAGG market does not exist for this coin pair (ARKER-USDT)
[ERROR] CCCAGG market does not exist for this coin pair (FRR-USDT)
[ERROR] CCCAGG market does not exist for this coin pair (MAKI-USDT)
[ERROR] CCCAGG market does not exist for this coin pair (SWASH-USDT)
[ERROR] CCCAGG market does not exist for this coin pair (NTVRK-USDT)
[ERROR] CCCAGG market does not exist for this coin pair (FTG-USDT)
[ERROR] CCCAGG market does not exist for this coin pair (ZKT-USDT)
[ERROR] CCCAGG market does not exist for this coin pair (PEL-USDT)
[ERROR] CCCAGG market does not exist for this coin pair (SOLR-USDT)
[ERROR] CCCAGG market does not exist for this coin pair (ILA-USDT)
[ERROR] CCCAGG market does not exist for this coin pair (CLH-USDT)
[ERROR] CCCAGG market does not exist for this coin pair (CREDI-USDT)
[ERROR] CCCAGG market does not exist for this coin pair (RACEFI-USDT)
[ERROR] CCCAGG market does not 

587

In [122]:
len(alts_list)

587

In [123]:
import joblib as jb
jb.dump(list(set(alts_list)), './data/alts_list.pkl')

['alts_list.pkl']

In [137]:
small_list = list(set(alts_list))
jb.dump(small_list, './data/small_list.pkl')
len(small_list)

585

In [140]:
# найдем биткойн
for i, item in enumerate(small_list):
    if item == 'BTC':
        print(i)

399


In [141]:
small_list[399]

'BTC'

In [125]:
btc = cc.get_historical_price_day('BTC', currency='USDT', limit=30)

In [126]:
btc[0]

{'time': 1660867200,
 'high': 23206.49,
 'low': 20788.6,
 'open': 23201.87,
 'volumefrom': 397496.38,
 'volumeto': 8646441203.59,
 'close': 20834.67,
 'conversionType': 'direct',
 'conversionSymbol': ''}

### Загрузка данных и проверка результатов

In [142]:
%%time
day30_dict = {ticker: cc.get_historical_price_day(ticker, currency='USDT', limit=30) for ticker in tqdm(small_list)}

  0%|          | 0/585 [00:00<?, ?it/s]

CPU times: user 58.1 s, sys: 7.54 s, total: 1min 5s
Wall time: 4min 19s


In [143]:
df_temp = pd.DataFrame(day30_dict).T.dropna().apply(lambda x: x[:])
for col in df_temp.columns:
    df_temp[col] = df_temp[col].apply(lambda x: x['close'])
df_temp.to_csv('./data/df30d_crypto.csv')


In [144]:
df_temp.shape

(585, 31)

In [145]:
df_temp.index

Index(['1GOLD', 'CFG', 'WTC', 'FLUX', 'AGLD', 'GMTT', 'USDP', 'BTG', 'NUT',
       'BHPC',
       ...
       'NXM', 'PAXG', 'XLM', 'RDN', 'EUL', 'TRAC', 'MUSE', 'INDI', 'ELA',
       'XHV'],
      dtype='object', length=585)

In [149]:
%%time
day3_dict = {ticker: cc.get_historical_price_hour(ticker, currency='USDT', limit=3*24) for ticker in tqdm(small_list)}
min60_dict = {ticker: cc.get_historical_price_minute(ticker, currency='USDT', limit=60) for ticker in tqdm(small_list)}

  0%|          | 0/585 [00:00<?, ?it/s]

  0%|          | 0/585 [00:00<?, ?it/s]

CPU times: user 1min 54s, sys: 15.2 s, total: 2min 9s
Wall time: 8min 45s


In [150]:
df_temp = pd.DataFrame(day3_dict).T.dropna().apply(lambda x: x[:])
for col in df_temp.columns:
    df_temp[col] = df_temp[col].apply(lambda x: x['close'])
df_temp.to_csv('./data/df3d_crypto.csv')

In [151]:
df_temp = pd.DataFrame(min60_dict).T.dropna().apply(lambda x: x[:])
for col in df_temp.columns:
    df_temp[col] = df_temp[col].apply(lambda x: x['close'])
df_temp.to_csv('./data/df60min_crypto.csv')

In [173]:
day30_data = pd.read_csv('./data/df30d_crypto.csv', header=0)
day3_data = pd.read_csv('./data/df3d_crypto.csv')
min60_data = pd.read_csv('./data/df60min_crypto.csv')

In [174]:
print(day30_data.shape)
print(day3_data.shape)
print(min60_data.shape)

(585, 32)
(585, 74)
(585, 62)


In [194]:
min60_data

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,1GOLD,56.0000,56.0000,56.0000,56.0000,56.0000,56.0000,56.0000,56.0000,56.0000,...,56.0000,56.0000,56.0000,56.0000,56.0000,56.0000,56.0000,56.0000,56.0000,56.0000
1,CFG,0.3387,0.3391,0.3386,0.3387,0.3389,0.3388,0.3389,0.3383,0.3385,...,0.3384,0.3387,0.3393,0.3392,0.3390,0.3389,0.3391,0.3389,0.3388,0.3391
2,WTC,0.2806,0.2805,0.2806,0.2809,0.2808,0.2809,0.2808,0.2809,0.2809,...,0.2785,0.2796,0.2791,0.2783,0.2785,0.2779,0.2778,0.2777,0.2764,0.2757
3,FLUX,0.9803,0.9784,0.9722,0.9665,0.9657,0.9668,0.9685,0.9686,0.9691,...,0.9732,0.9770,0.9769,0.9761,0.9740,0.9746,0.9746,0.9745,0.9708,0.9665
4,AGLD,0.3340,0.3340,0.3340,0.3333,0.3340,0.3340,0.3340,0.3339,0.3339,...,0.3320,0.3320,0.3320,0.3310,0.3311,0.3311,0.3311,0.3311,0.3289,0.3249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,TRAC,0.1896,0.1896,0.1896,0.1897,0.1897,0.1912,0.1913,0.1911,0.1912,...,0.1895,0.1895,0.1895,0.1895,0.1906,0.1904,0.1895,0.1895,0.1872,0.1872
581,MUSE,8.3100,8.3660,8.3100,8.3290,8.3100,8.3040,8.3450,8.3040,8.3030,...,8.3800,8.3060,8.3060,8.3060,8.3050,8.3380,8.3050,8.3190,8.3050,8.2840
582,INDI,0.1324,0.1324,0.1324,0.1324,0.1324,0.1323,0.1324,0.1325,0.1325,...,0.1324,0.1328,0.1328,0.1323,0.1323,0.1323,0.1323,0.1323,0.1327,0.1327
583,ELA,1.5080,1.5090,1.5090,1.5080,1.5090,1.5100,1.5100,1.5080,1.5100,...,1.5090,1.5100,1.5080,1.5090,1.5100,1.5100,1.5100,1.5100,1.5090,1.5090
